# PPV VISUALS

In [5]:
import os, glob, psutil

import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
import matplotlib.pyplot as pl
import matplotlib.colors as mc
from matplotlib.patches import Rectangle
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from mpl_toolkits.mplot3d import axes3d, art3d, proj3d

from scipy.interpolate import NearestNDInterpolator
from numpy import linspace, array, logspace, sin, cos, pi, arange, sqrt, arctan2, arccos
from coords import *
from adjustText import adjust_text
import matplotlib.patheffects as PathEffects
import matplotlib.patches as mpatches
from astropy.coordinates import Angle, SkyCoord, Longitude
from astropy.nddata.utils import Cutout2D
import astropy.units as u
from astropy.wcs.utils import pixel_to_skycoord
import astropy.io.fits as pyfits 
from astropy.convolution import convolve_fft, Gaussian2DKernel
import gc
gc.enable()
from regions import Regions
from spectral_cube import SpectralCube

#import skimage as ski
#import skimage.measure as measure
#from skimage.draw import polygon2mask


In [6]:
mask_file_list=[]
ID_num = []

mask_path = './inflow_EVF_masked_cubes/'# glob.glob() return a list of file name with specified pathname
for file in glob.glob(mask_path + '*.fits', recursive=True):
    mask_file_list.append(os.path.basename(file)) 
    ID_num.append(int(os.path.basename(file).split('.fits',1)[0].split('_',2)[2]))


Drive_path = '/Users/danilipman/Google Drive/Shared drives/ACES/Paper Sprints/Inflow_EVFs/DATA/Identification/TILES_TABLES/'
EVF_tab = Table.read(Drive_path + 'Filtered_EVFs_table.ecsv')


scouse_fits = pd.read_csv('/Users/danilipman/Documents/Research/UConn/CMZ_datasets/final_cmz_scouse_hnco_fits.csv',
              usecols=[0,1,2,3,5,7], names=['n', 'l', 'b', 'amp', 'velocity', 'FWHM'],
              sep="\s+")

l   = scouse_fits['l'].values
b   = scouse_fits['b'].values
v   = scouse_fits['velocity'].values
amp = scouse_fits['amp'].values

<>:16: SyntaxWarning: invalid escape sequence '\s'
<>:16: SyntaxWarning: invalid escape sequence '\s'
/var/folders/4b/_cshhy012yj8k_187jxt881r0000gn/T/ipykernel_76636/3025133913.py:16: SyntaxWarning: invalid escape sequence '\s'
  sep="\s+")


In [7]:
## Read in the line ratio statistics from Claire! ##
ratio_stats = '/Users/danilipman/Google Drive/Shared drives/ACES/Paper Sprints/Inflow_EVFs/DATA/Chemistry/RatioMap_Stats_AllEVFs.txt'
X1,SO21_CS21_Average,SO21_CS21_Median,SO21_CS21_Std_Dev,HNCO_SO21_Average,HNCO_SO21_Median,HNCO_SO21_Std_Dev,HNCO_H13CN_Average,HNCO_H13CN_Median,HNCO_H13CN_Std_Dev,HN13C_SO21_Average,HN13C_SO21_Median,HN13C_SO21_Std_Dev,HNCO_H13COp_Average,HNCO_H13COp_Median,HNCO_H13COp_Std_Dev,SO21_H13CN_Average,SO21_H13CN_Median,SO21_H13CN_Std_Dev,SO21_SO32_Average,SO21_SO32_Median,SO21_SO32_Std_Dev,HC3N_SiO21_Average,HC3N_SiO21_Median,HC3N_SiO21_Std_Dev,SO32_SiO21_Average,SO32_SiO21_Median,SO32_SiO21_Std_Dev,HC3N_H13COp_Average,HC3N_H13COp_Median,HC3N_H13COp_Std_Dev,HN13C_CS21_Average,HN13C_CS21_Median,HN13C_CS21_Std_Dev,HN13C_H13COp_Average,HN13C_H13COp_Median,HN13C_H13COp_Std_Dev,HNCO_CS21_Average,HNCO_CS21_Median,HNCO_CS21_Std_Dev,HNCO_HC3N_Average,HNCO_HC3N_Median,HNCO_HC3N_Std_Dev,HNCO_HN13C_Average,HNCO_HN13C_Median,HNCO_HN13C_Std_Dev,HNCO_SO32_Average,HNCO_SO32_Median,HNCO_SO32_Std_Dev,HN13C_SiO21_Average,HN13C_SiO21_Median,HN13C_SiO21_Std_Dev,H13COp_CS21_Average,H13COp_CS21_Median,H13COp_CS21_Std_Dev,HN13C_SO32_Average,HN13C_SO32_Median,HN13C_SO32_Std_Dev,HN13C_H13CN_Average,HN13C_H13CN_Median,HN13C_H13CN_Std_Dev,SO21_H13COp_Average,SO21_H13COp_Median,SO21_H13COp_Std_Dev,SiO21_H13COp_Average,SiO21_H13COp_Median,SiO21_H13COp_Std_Dev,HC3N_CS21_Average,HC3N_CS21_Median,HC3N_CS21_Std_Dev,SO32_H13COp_Average,SO32_H13COp_Median,SO32_H13COp_Std_Dev,SiO21_CS21_Average,SiO21_CS21_Median,SiO21_CS21_Std_Dev,HC3N_SO32_Average,HC3N_SO32_Median,HC3N_SO32_Std_Dev,H13COp_H13CN_Average,H13COp_H13CN_Median,H13COp_H13CN_Std_Dev,HC3N_HN13C_Average,HC3N_HN13C_Median,HC3N_HN13C_Std_Dev,SO21_SiO21_Average,SO21_SiO21_Median,SO21_SiO21_Std_Dev,SiO21_H13CN_Average,SiO21_H13CN_Median,SiO21_H13CN_Std_Dev,HC3N_SO21_Average,HC3N_SO21_Median,HC3N_SO21_Std_Dev,H13CN_CS21_Average,H13CN_CS21_Median,H13CN_CS21_Std_Dev,SO32_H13CN_Average,SO32_H13CN_Median,SO32_H13CN_Std_Dev,SO32_CS21_Average,SO32_CS21_Median,SO32_CS21_Std_Dev,HC3N_H13CN_Average,HC3N_H13CN_Median,HC3N_H13CN_Std_Dev,HNCO_SiO21_Average,HNCO_SiO21_Median,HNCO_SiO21_Std_Dev = np.genfromtxt(ratio_stats, dtype=float,skip_header=1,filling_values = -99.99, unpack=True)
data = np.loadtxt(ratio_stats, dtype=str,skiprows=1, unpack=True) #for text
evf_name = data[0]

#match up ID numbers 
ratio_evf_ID = []
for i, evf in enumerate(evf_name):
    ratiol,ratiob = evf.split('_',1)
    ratio_lcoord,  ratio_bcoord= np.round(float(ratiol.split('l')[1]),3),float(ratiob.split('b')[1])

    matchcoord = np.where((EVF_tab['l']==ratio_lcoord) & (EVF_tab['b']==ratio_bcoord))[0]
    ratio_evf_ID.append(int(EVF_tab['ID Number'][matchcoord][0]))
    


In [8]:
stvec = {}
for i, maskedcube in list(enumerate(mask_file_list)):
    ID_num = int(os.path.basename(maskedcube).split('.fits',1)[0].split('_',2)[2])
    striat = []
    data = pyfits.open(mask_path+maskedcube) 
    cube = SpectralCube.read(data)      
    data.close()   
    del data
    gc.collect()

    cube_masked = cube
    
    v1, b1, l1 = cube_masked.world[~np.ma.masked_invalid(cube_masked).mask]
    l1 = Longitude(l1, unit='deg', wrap_angle=180*u.deg)

    striat.append(np.dstack([l1.value, b1.value, v1.value]))

    stvec[f'{ID_num}'] = striat



In [9]:
##filter table by 20pc projection near SgrA*

infall_mask_20 = (EVF_tab['l']**2 + EVF_tab['b']**2) <= np.rad2deg(np.arcsin(20/8200))**2
infall_mask_40 = ((EVF_tab['l']**2 + EVF_tab['b']**2) > np.rad2deg(np.arcsin(20/8200))**2 )& ((EVF_tab['l']**2 + EVF_tab['b']**2) <= np.rad2deg(np.arcsin(40/8200))**2)
infall_mask_50 = ((EVF_tab['l']**2 + EVF_tab['b']**2) > np.rad2deg(np.arcsin(40/8200))**2 )& ((EVF_tab['l']**2 + EVF_tab['b']**2) <= np.rad2deg(np.arcsin(50/8200))**2)

infall_tab_20 = EVF_tab[infall_mask_20]
infall_tab_40 = EVF_tab[infall_mask_40]
infall_tab_50 = EVF_tab[infall_mask_50]


In [10]:
infall_tab_50.write("candidates.csv", overwrite=True)


In [11]:
### ELLIPSE ###
from coords import *

def rotate(x,y,theta):
    xprime = x*cos(theta) - y*sin(theta)
    yprime = x*sin(theta) + y*cos(theta)
    return xprime, yprime


class Ring:

    def __init__(self,t,a,b,z,v0,theta,xyzsun,vxyzsun,alpha=0.4):
        self.t     = t
        self.a     = a
        self.b     = b
        self.z0    = z
        self.v0    = v0
        self.theta = theta
        self.x     = a*cos(t)
        self.y     = -b*sin(t)
        self.z     = self.z0*sin(-2*t + alpha)
        self.R     = sqrt(self.x**2+self.y**2)
        self.phi   = -arctan2(self.y,self.x)
        self.ephix = -sin(self.phi) # unit vector parallel to circle
        self.ephiy = -cos(self.phi) # unit vector parallel to circle
        norm       = sqrt((a*sin(t))**2+(b*cos(t))**2)
        self.ex    = -a*sin(t)/norm # unit vector parellel to ellipse
        self.ey    = -b*cos(t)/norm  # unit vector parallel to ellipse
        self.cosalpha = self.ex*self.ephix + self.ey*self.ephiy
        self.vphi  = self.R[0]*self.v0/self.R # assume conservation of angular momentum
        self.v     = self.vphi/self.cosalpha # total speed along the orbit
        self.vx    = +self.v*self.ex
        self.vy    = +self.v*self.ey
        self.vz    = np.zeros(t.size)
        self.x,self.y   = rotate(self.x,self.y,theta)
        self.vx,self.vy = rotate(self.vx,self.vy,theta)
        self.X,self.Y,self.Z,self.vX,self.Vy,self.vZ = xyz2XYZ(self.x,self.y,self.z,self.vx,self.vy,self.vz,xyzsun[0],xyzsun[1],xyzsun[2],vxyzsun[0],vxyzsun[1],vxyzsun[2])
        # Convert Sgr A* offset from degrees to radians
        l_offset = np.radians(0.05)
        b_offset = np.radians(-0.0462)
        x_offset = Rsun*l_offset
        y_offset = 0
        z_offset = Rsun*b_offset
        # Calculate galactic coordinates accounting for Sgr A* offset
        self.l,self.b,self.r,self.vl,self.vb,self.vr = xyz2lbr(
            self.x,self.y,self.z,self.vx,self.vy,self.vz,
            xyzsun[0],xyzsun[1],xyzsun[2],vxyzsun[0],vxyzsun[1],vxyzsun[2])
        self.l += l_offset
        self.b += b_offset
        self.x += x_offset
        self.y += y_offset
        self.z += z_offset
        self.mu_l, self.mu_b = vlb_2_mulb(self.r, self.vl*100, self.vb*100)
        self.mu_l, self.mu_b = vlb_2_mulb(self.r,self.vl*100,self.vb*100)


#############################
# define sun position & velocity
#############################

xsun  = 0.0
ysun  = -8.1
zsun  = 0.0
vxsun = -2.2
vysun = 0.0
vzsun = 0.0
xyzsun  = [xsun, ysun, zsun ]
vxyzsun = [vxsun,vysun,vzsun]
phisun  = arctan2(ysun,xsun)

#############################
# define rings
#############################

a_rings = 0.09 # x axis of the rings in kpc
b_rings = 0.055 # y axis of the rings in kpc
z_rings = 0.0125 # height of the ring above/below plane in kpc
v_rings = 130.0 # tangential velocity at initial point
theta_rings = np.radians(25) # inclination of the rings
dphi = np.radians(1)
phi  = arange(phisun+dphi/2,phisun+2*pi,dphi)
Rings = Ring(phi,a_rings,b_rings,z_rings,v_rings,theta_rings,xyzsun,vxyzsun)



# HNCO/SiO

In [ ]:
rationorm = mc.TwoSlopeNorm(vmin=0, vcenter = 0.5 , vmax=1)
cmap='jet'

fig = plt.figure(figsize=(15,12),dpi=100)
ax = fig.add_subplot(111, projection='3d', rasterized=True)
axes0.set_box_aspect(aspect=(1.5, 1, 1), zoom=1.5)

fig.set_facecolor('white')
axes0.set_facecolor('white')
axes0.grid(False)
axes0.xaxis.pane.set_edgecolor('black')
axes0.yaxis.pane.set_edgecolor('black')
ax.zaxis.pane.set_edgecolor('black')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.grid(False) # Hide grid lines
#change spine position
ax.zaxis._axinfo['juggled'] = (1,2,0)

ax.scatter(-0.056, -0.046, 0, marker = "*", s=20, color='k') #plot SgrA*



ax.scatter(l, b, v, c=amp, cmap='Grays', marker='o', s=1, facecolor='1', norm=mpl.colors.LogNorm(), alpha=0.1, zorder=10)
ax.plot(np.degrees(Rings.l),np.degrees(Rings.b), Rings.vr, ls='-', color='k', linewidth=2, zorder=2, alpha=0.4)

print(f"Total CPU Usage before enumerate: {psutil.cpu_percent(interval=1)}%")
for reg in enumerate(list(stvec.keys())):
    for k in stvec[reg[1]]:
        if int(reg[1]) in ratio_evf_ID:
            plotc = float(HNCO_SiO21_Median[int(np.where(int(reg[1]) == np.array(ratio_evf_ID) )[0][0])])
            l1, b1, v1 = k[:,:,0][0], k[:,:,1][0],k[:,:,2][0]
            sc = ax.scatter(l1, b1, v1, c=np.full(len(l1),plotc), cmap=cmap, norm=rationorm, marker='o', s=0.5, alpha=0.1, zorder=1)
            del l1, b1, v1, reg, k, sc 
            gc.collect()      
  
        else:
            plotc = 'k'
            l1, b1, v1 = k[:,:,0][0], k[:,:,1][0],k[:,:,2][0]
            sc = ax.scatter(l1, b1, v1, color=plotc, marker='o', s=0.5, alpha=0.1, zorder=1)
            del l1, b1, v1, reg, k, sc 
            gc.collect()   
 
print(f"Total CPU Usage after enumerate: {psutil.cpu_percent(interval=1)}%")

        
sm = plt.cm.ScalarMappable(cmap=cmap, norm=rationorm)
sm.set_array([])
print(f"Total CPU Usage set sm: {psutil.cpu_percent(interval=1)}%")

cbar = fig.colorbar(sm, ax=ax, location='bottom')
cbar.set_label('HNCO_SiO21_Median')

# Set the azimuth and elevation angles
ax.view_init(azim=-85, elev=5)
ax.dist = 13
print(f"Total CPU Usage set view_init: {psutil.cpu_percent(interval=1)}%")

ax.set_xticks(ticks = [-1.0, -0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
ax.tick_params(axis='both', which='major', labelsize=14)
ax.tick_params(axis='both', which='minor', labelsize=14)
ax.set_xlim(1,-1)
ax.set_ylim(-0.3,0.3)
ax.set_zlim(-150.,150.)
ax.set_xlabel('GLON (deg)', labelpad=20, fontsize=14)
ax.set_ylabel('GLAT (deg)', labelpad=20, fontsize=14)
ax.set_zlabel('Velocity (km/s)', labelpad=20, fontsize=14, rotation=90)
ax.zaxis.set_rotate_label(False)
print(f"Total CPU Usage set ax.zaxis: {psutil.cpu_percent(interval=1)}%")


plt.show()
print(f"Total CPU Usage after show: {psutil.cpu_percent(interval=1)}%")

#plt.savefig('./HNCO_SiO_Median_PPV_lb.pdf', bbox_inches='tight')
#print(f"Total CPU Usage after savefig: {psutil.cpu_percent(interval=1)}%")

plt.cla()
plt.clf()
plt.close('all')
print(f"Total CPU Usage after clf: {psutil.cpu_percent(interval=1)}%")


del fig, ax, sm
gc.collect()
print(f"Total CPU Usage after gc: {psutil.cpu_percent(interval=1)}%")





Total CPU Usage before enumerate: 11.9%
Total CPU Usage after enumerate: 12.9%
Total CPU Usage set sm: 14.1%
Total CPU Usage set view_init: 10.7%
Total CPU Usage set ax.zaxis: 12.2%
Total CPU Usage after savefig: 86.2%
Total CPU Usage after clf: 84.5%
Total CPU Usage after gc: 81.6%


# Gridspec double plot

In [ ]:
# Setup our Grid Speced plots
# Include an extra column between z and zoom for spacing
n_rows = 2
n_cols = 4
fontsize = 20
# 18 is more or less arbitrary,
# the ratio is important so that the imshow plots have 1:1 aspect ratio
width = 40
height_ratio = 0.5
fig = plt.figure(figsize=(width, width * height_ratio))
width_ratios = [0.0, 2.0, 0.0, 2.5 ]
height_ratios = [2.0, 0.1]
gs1 = fig.add_gridspec(
    n_rows, n_cols,
    width_ratios = width_ratios,
    height_ratios = height_ratios,
    wspace = 0.5,
    hspace = 0.3,
)
axes0 = fig.add_subplot(gs1[0, 1], projection = '3d', rasterized=True)
axes1 = fig.add_subplot(gs1[0, 3], rasterized=True)
colorbar_ax = fig.add_subplot(gs1[1, :])



### 3D plot 

rationorm = mc.TwoSlopeNorm(vmin=0, vcenter = 0.5 , vmax=1)
cmap='magma_r'

axes0.set_box_aspect(aspect=(1.5, 1, 1.5), zoom=1.8)

fig.set_facecolor('white')
axes0.set_facecolor('white')
axes0.grid(False)
axes0.xaxis.pane.set_edgecolor('black')
axes0.yaxis.pane.set_edgecolor('black')
axes0.zaxis.pane.set_edgecolor('black')
axes0.xaxis.pane.fill = False
axes0.yaxis.pane.fill = False
axes0.zaxis.pane.fill = False
axes0.grid(False) # Hide grid lines
#change spine position
axes0.zaxis._axinfo['juggled'] = (1,2,0)

axes0.scatter(-0.056, -0.046, 0, marker = "*", s=20, color='k') #plot SgrA*
axes0.scatter(l, b, v, c=amp, cmap='Grays', marker='o', s=1, facecolor='1', norm=mpl.colors.LogNorm(), alpha=0.1, zorder=10)
axes0.plot(np.degrees(Rings.l),np.degrees(Rings.b), Rings.vr, ls='-', color='k', linewidth=2, zorder=2, alpha=0.4)


axes0.view_init(azim=-85, elev=5)
axes0.dist = 13

axes0.set_xticks(ticks = [-1.0, -0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
axes0.set_yticks(ticks = [-0.3, -0.2, -0.1, 0 , 0.1, 0.2, 0.3])
# Rotate Y-axis tick labels
for tlabel in axes0.get_yticklabels():
    tlabel.set_rotation(30) # Rotate by -30 degrees


axes0.tick_params(axis='both', which='major', labelsize=fontsize)

axes0.tick_params(axis='both', which='minor', labelsize=fontsize)
axes0.set_xlim(1,-1)
axes0.set_ylim(-0.3,0.3)
axes0.set_zlim(-150.,150.)
axes0.set_xlabel('GLON (deg)', labelpad=10, fontsize=fontsize)
axes0.set_ylabel('GLAT (deg)', labelpad=10, fontsize=fontsize)
axes0.set_zlabel('Velocity (km/s)', labelpad=10, fontsize=fontsize, rotation=90)
axes0.zaxis.set_rotate_label(False)




#### LB plot 
axes1.set_facecolor('white')
axes1.grid(False)
axes1.grid(False) # Hide grid lines
#cha1ge spine position
axes1.scatter(-0.056, -0.046, marker = "*", s=20, color='k') #plot SgrA*
axes1.scatter(l, b, c=amp, cmap='Grays', marker='o', s=1, facecolor='1', norm=mpl.colors.LogNorm(), alpha=0.5, zorder=10)
axes1.plot(np.degrees(Rings.l),np.degrees(Rings.b),  ls='-', color='k', linewidth=2, zorder=2, alpha=0.7)

axes1.set_xticks(ticks = [-1.0, -0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
axes1.tick_params(axis='both', which='major', labelsize=fontsize)
axes1.tick_params(axis='both', which='minor', labelsize=fontsize)
axes1.set_xlim(1,-0.7)
axes1.set_ylim(-0.3,0.21)
axes1.set_xlabel('GLON (deg)', labelpad=20, fontsize=fontsize)
axes1.set_ylabel('GLAT (deg)', labelpad=20, fontsize=fontsize)




for reg in enumerate(list(stvec.keys())):
    for k in stvec[reg[1]]:
        if int(reg[1]) in ratio_evf_ID:
            plotc = float(HNCO_SiO21_Median[int(np.where(int(reg[1]) == np.array(ratio_evf_ID) )[0][0])])
            l1, b1, v1 = k[:,:,0][0], k[:,:,1][0],k[:,:,2][0]
            sc0 = axes0.scatter(l1, b1, v1, c=np.full(len(l1),plotc), cmap=cmap, norm=rationorm, marker='o', s=10, alpha=0.1, zorder=1)
            sc1 = axes1.scatter(l1, b1, c=np.full(len(l1),plotc), cmap=cmap, norm=rationorm, marker='o', s=10, alpha=0.1, zorder=1)
            del l1, b1, v1, reg, k, sc1,sc0 
            gc.collect()      
  
        else:
            continue
            """plotc = 'k'
            l1, b1, v1 = k[:,:,0][0], k[:,:,1][0],k[:,:,2][0]
            sc0 = axes0.scatter(l1, b1, v1, color=plotc, marker='o', s=10, alpha=0.1, zorder=1)
            sc1 = axes1.scatter(l1, b1, color=plotc, marker='o', s=10, alpha=0.1, zorder=1)
            del l1, b1, v1, reg, k, sc1,sc0 
            gc.collect()   """





 
sm = fig.colorbar(
    mpl.cm.ScalarMappable(cmap=cmap, norm=rationorm),
    cax=colorbar_ax,
    spacing='proportional',
    orientation='horizontal',
)

sm.ax.tick_params(labelsize=fontsize)  # set your label size here
sm.set_label('HNCO/SiO21 Median', labelpad=10, fontsize=30)
print(f"Total CPU Usage set sm: {psutil.cpu_percent(interval=1)}%")



plt.savefig('./HNCO_SiO_Median_PPV_double.pdf', bbox_inches='tight')
print(f"Total CPU Usage after savefig: {psutil.cpu_percent(interval=1)}%")

plt.cla()
plt.clf()
plt.close('all')
print(f"Total CPU Usage after clf: {psutil.cpu_percent(interval=1)}%")


del fig, axes0, axes1, sm
gc.collect()
print(f"Total CPU Usage after gc: {psutil.cpu_percent(interval=1)}%")



/var/folders/4b/_cshhy012yj8k_187jxt881r0000gn/T/ipykernel_76636/2022075652.py:47: UserWarning: You passed both c and facecolor/facecolors for the markers. c has precedence over facecolor/facecolors. This behavior may change in the future.
  axes0.scatter(l, b, v, c=amp, cmap='Grays', marker='o', s=1, facecolor='1', norm=mpl.colors.LogNorm(), alpha=0.1, zorder=10)
/var/folders/4b/_cshhy012yj8k_187jxt881r0000gn/T/ipykernel_76636/2022075652.py:81: UserWarning: You passed both c and facecolor/facecolors for the markers. c has precedence over facecolor/facecolors. This behavior may change in the future.
  axes1.scatter(l, b, c=amp, cmap='Grays', marker='o', s=1, facecolor='1', norm=mpl.colors.LogNorm(), alpha=0.5, zorder=10)


Total CPU Usage set sm: 23.4%
Total CPU Usage after savefig: 74.1%
Total CPU Usage after clf: 77.7%
Total CPU Usage after gc: 74.9%


# HNCO/HC3N

In [ ]:
rationorm = mc.TwoSlopeNorm(vmin=0, vcenter = float(np.median(HNCO_HC3N_Median)), vmax=1)
cmap='jet'

fig = plt.figure(figsize=(15,12), dpi=100)
ax = fig.add_subplot(111, projection='3d', rasterized=True)
ax.set_box_aspect(aspect=(1.5, 1, 1), zoom=1.5)


fig.set_facecolor('white')
ax.set_facecolor('white')
ax.grid(False)
ax.xaxis.pane.set_edgecolor('black')
ax.yaxis.pane.set_edgecolor('black')
ax.zaxis.pane.set_edgecolor('black')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.grid(False) # Hide grid lines
#change spine position
ax.zaxis._axinfo['juggled'] = (1,2,0)

ax.scatter(-0.056, -0.046, 0, marker = "*", s=20, color='k') #plot SgrA*



ax.scatter(l, b, v, c=amp, cmap='Grays', marker='o', s=1, facecolor='1', norm=mpl.colors.LogNorm(), alpha=0.1, zorder=10)
ax.plot(np.degrees(Rings.l),np.degrees(Rings.b), Rings.vr, ls='-', color='k', linewidth=2, zorder=2, alpha=0.4)


for reg in enumerate(list(stvec.keys())):
    for k in stvec[reg[1]]:
        if int(reg[1]) in ratio_evf_ID:
            plotc = float(HNCO_HC3N_Median[int(np.where(int(reg[1]) == np.array(ratio_evf_ID) )[0][0])])
            l1, b1, v1 = k[:,:,0][0], k[:,:,1][0],k[:,:,2][0]
            ax.scatter(l1, b1, v1, c=np.full(len(l1),plotc), cmap=cmap, norm=rationorm, marker='o', s=0.5, alpha=0.1, zorder=1)
            del l1, b1, v1, reg, k, fig, ax
            gc.collect()
                        
        else:
            plotc = 'k'
            l1, b1, v1 = k[:,:,0][0], k[:,:,1][0],k[:,:,2][0]
            ax.scatter(l1, b1, v1, color=plotc, marker='o', s=0.5, alpha=0.1, zorder=1)
            del l1, b1, v1, reg, k, fig, ax
            gc.collect()

        
sm = plt.cm.ScalarMappable(cmap=cmap, norm=rationorm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, location='bottom')
cbar.set_label('HNCO_HC3N_Median')

# Set the azimuth and elevation angles
ax.view_init(azim=-85, elev=90)
ax.dist = 13

ax.set_xticks(ticks = [-1.0, -0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
ax.tick_params(axis='both', which='major', labelsize=14)
ax.tick_params(axis='both', which='minor', labelsize=14)
ax.set_xlim(1,-1)
ax.set_ylim(-0.3,0.3)
ax.set_zlim(-150.,150.)
ax.set_xlabel('GLON (deg)', labelpad=20, fontsize=14)
ax.set_ylabel('GLAT (deg)', labelpad=20, fontsize=14)
ax.set_zlabel('Velocity (km s^-1)', labelpad=20, fontsize=14, rotation=90)
ax.zaxis.set_rotate_label(False)



plt.savefig('./HNCO_HC3N_Median_PPV_lb.pdf', bbox_inches='tight')

plt.close(fig)






In [ ]:
fig = plt.figure(figsize=(15,12), dpi=100)
ax = fig.add_subplot(111, projection='3d', rasterized=True)
ax.set_box_aspect(aspect=(1.5, 1, 1), zoom=1.5)


fig.set_facecolor('white')
ax.set_facecolor('white')
ax.grid(False)
ax.xaxis.pane.set_edgecolor('black')
ax.yaxis.pane.set_edgecolor('black')
ax.zaxis.pane.set_edgecolor('black')
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.grid(False) # Hide grid lines
#change spine position
ax.zaxis._axinfo['juggled'] = (1,2,0)

ax.scatter(-0.056, -0.046, 0, marker = "*", s=20, color='k') #plot SgrA*



ax.scatter(l, b, v, c=amp, cmap='Grays', marker='o', s=1, facecolor='1', norm=mpl.colors.LogNorm(), alpha=0.1, zorder=10)
ax.plot(np.degrees(Rings.l),np.degrees(Rings.b), Rings.vr, ls='-', color='k', linewidth=2, zorder=2, alpha=0.4)


for reg in enumerate(list(stvec.keys())):
    for k in stvec[reg[1]]:
        if int(reg[1]) in infall_tab_20['ID Number']:
            plotc = 'red'
            
        elif int(reg[1]) in infall_tab_40['ID Number']:
            plotc = 'orange'

        elif int(reg[1]) in infall_tab_50['ID Number']:
            plotc = 'lime'

        else:
            plotc = 'k'
        
        l1, b1, v1 = k[:,:,0][0], k[:,:,1][0],k[:,:,2][0]
        ax.scatter(l1, b1, v1, color=plotc, marker='o', s=0.5, alpha=0.1, zorder=1)
        

# Set the azimuth and elevation angles
ax.view_init(azim=-85, elev=5)
ax.dist = 13

ax.set_xticks(ticks = [-1.0, -0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
ax.tick_params(axis='both', which='major', labelsize=14)
ax.tick_params(axis='both', which='minor', labelsize=14)
ax.set_xlim(1,-1)
ax.set_ylim(-0.3,0.3)
ax.set_zlim(-150.,150.)
ax.set_xlabel('GLON (deg)', labelpad=20, fontsize=14)
ax.set_ylabel('GLAT (deg)', labelpad=20, fontsize=14)
ax.set_zlabel('Velocity (km s^-1)', labelpad=20, fontsize=14, rotation=90)
ax.zaxis.set_rotate_label(False)

plt.show()
plt.close()

del l1, b1, v1, reg, k, fig, ax
gc.collect()
